RALLY:<br>
- SRV_TEAM_ID: serving team<br>
- W_TEAM_ID: who wins the rally<br>
- TO_IND: timeout indicator records when a timeout is taken<br>
- TO_TEAM_ID: team taking the timeout<br>
- RALLY_ID: primary key<br>
- POINT_NBR: order that rallies are played<br> 

GAME:<br>
- GAME_ID: will connect RALLY and GAME (>=1 for a match)<br> 

approach:<br>
- SRV_TEAM_ID + RTN_TEAM_ID will show who is serving<br>
- W_TEAM_ID then shows who actually won the rally<br>
- ordered by POINT_NBR (within a game)<br> 
- prior to a row where the TO_IND is ‘Y’<br>

In [3]:
import pandas as pd
from collections import Counter
import numpy as np

In [4]:
df_game = pd.read_csv('csvs/game.csv')[['match_id','game_id','game_nbr','score_w','score_l','w_team_id','l_team_id','skill_lvl']]
df_game['game_id'] = [int(x[1:]) for x in df_game.game_id]
df_game['match_id'] = [int(x[1:]) for x in df_game.match_id]
df_game = df_game.sort_values(['match_id','game_id']).reset_index(drop=True)
print('GAME:\n ')
print(df_game.head(2).to_string(index=False))
print('\n...last rows:\n')
print(df_game.tail(2).to_string(index=False))

GAME:
 
 match_id  game_id  game_nbr  score_w  score_l w_team_id l_team_id skill_lvl
        1        1         1       12       10        T2        T1       Pro
        1        2         2       11        7        T1        T2       Pro

...last rows:

 match_id  game_id  game_nbr  score_w  score_l w_team_id l_team_id skill_lvl
       25       42         3       13       11       T39       T40       5.0
       25       43         2       11        5       T40       T39       5.0


In [5]:
df_rally = pd.read_csv('csvs/rally.csv')
df_rally = df_rally[[
    'match_id', 'game_id', 'rally_id', 'point_nbr', 'srv_team_id',
    'rtrn_team_id', 'w_team_id', 'to_ind', 'to_team_id', 'ending_type'
]]
df_rally['match_id'] = [int(x[1:]) for x in df_rally.match_id]
df_rally['game_id'] = [int(x[1:]) for x in df_rally.game_id]
df_rally['rally_id'] = [int(x[1:]) for x in df_rally.rally_id]
df_rally = df_rally.sort_values(
    ['match_id', 'game_id', 'rally_id', 'point_nbr']).reset_index(drop=True)
print('RALLY:\n')
print((df_rally.head(2).to_string(index=False)))
print('\n...last rows:\n')
print((df_rally.tail(2).to_string(index=False)))

RALLY:

 match_id  game_id  rally_id  point_nbr srv_team_id rtrn_team_id w_team_id to_ind to_team_id ending_type
        1        1         1          1          T1           T2        T2      N        NaN       Other
        1        1         2          2          T2           T1        T2      N        NaN      Winner

...last rows:

 match_id  game_id  rally_id  point_nbr srv_team_id rtrn_team_id w_team_id to_ind to_team_id ending_type
       25       43      2042         34         T39          T40       T40      N        NaN      Winner
       25       43      2043         35         T40          T39       T40      N        NaN      Winner


In [6]:
timeout_games = {}
for i in range(1, 1 + len(df_game)):
    game = df_rally[df_rally.game_id == i]
    timeout_games[i] = 'Y' in list(game.to_ind)

In [7]:
print(f'games without timeouts: {[x[0] for x in list(filter(lambda x:x[1] == False, timeout_games.items()))]}')

games without timeouts: [26, 32, 33, 34, 36, 41, 43]


In [8]:
maxrally = np.max(list(Counter(df_rally.game_id).values()))
print(f'most rallies({maxrally}):  game #{int(df_rally.game_id[df_rally.point_nbr == maxrally])}')

most rallies(71):  game #35


In [9]:
df_numrallys = df_rally[['game_id','point_nbr']].groupby(['game_id'],axis=0).max()

In [10]:
print(f"min # rallies: {int(df_numrallys.min())}")
print(f"avg # rallies: {float(df_numrallys.mean().round(1))}")
print(f"med # rallies: {int(df_numrallys.median())}")
print(f"max # rallies: {int(df_numrallys.max())}")

min # rallies: 20
avg # rallies: 47.5
med # rallies: 48
max # rallies: 71


In [11]:
for i in range(1, 1 + len(df_game)):
    one_game = df_rally[df_rally.game_id == i]
    todict = list(Counter(one_game.to_ind).items())
    if len(todict) == 1:
        print(f'game {int(i<10)*" "}#{i}: ')
    else:
        print(f'game {int(i<10)*" "}#{i}: {todict[1][1]}')

game  #1: 4
game  #2: 3
game  #3: 1
game  #4: 3
game  #5: 1
game  #6: 2
game  #7: 2
game  #8: 1
game  #9: 2
game #10: 1
game #11: 1
game #12: 1
game #13: 1
game #14: 1
game #15: 3
game #16: 1
game #17: 1
game #18: 1
game #19: 3
game #20: 1
game #21: 2
game #22: 2
game #23: 2
game #24: 1
game #25: 1
game #26: 
game #27: 1
game #28: 2
game #29: 1
game #30: 2
game #31: 1
game #32: 
game #33: 
game #34: 
game #35: 3
game #36: 
game #37: 1
game #38: 2
game #39: 2
game #40: 2
game #41: 
game #42: 1
game #43: 


In [26]:
numtos = []
numrallys = []
for i in range(1, 1 + len(df_game)):
    one_game = df_rally[df_rally.game_id == i]
    todict = list(Counter(one_game.to_ind).items())
    if len(todict) == 1:
        numtos.append(0)
    else:
        numtos.append(todict[1][1])
    numrallys.append(int(one_game.point_nbr.max()))

In [27]:
df_metrx = pd.DataFrame({'game_id':list(range(1,1+len(df_game.game_id))),'timeouts':numtos,'rallys':numrallys})
df_metrx['avg_unbroken'] = round(df_metrx.rallys / (1 + df_metrx.timeouts), 1)
df_metrx

,game_id,timeouts,rallys,avg_unbroken
0,1,4,63,12.6
1,2,3,54,13.5
2,3,1,38,19.0
3,4,3,60,15.0
4,5,1,37,18.5
5,6,2,43,14.3
6,7,2,41,13.7
7,8,1,53,26.5
8,9,2,54,18.0
9,10,1,33,16.5


In [14]:
df_avgunbrk = df_metrx[['timeouts','avg_unbroken']].groupby('timeouts').mean().round(1)
df_avgunbrk

,avg_unbroken
timeouts,
0,36.9
1,23.9
2,15.8
3,14.6
4,12.6


In [15]:
df_metrx.sort_values('avg_unbroken', ascending=False)

,game_id,timeouts,rallys,avg_unbroken
33,33,0,62,62.0
32,32,0,45,45.0
25,25,0,38,38.0
42,42,0,35,35.0
16,16,1,67,33.5
19,19,1,67,33.5
26,26,1,65,32.5
35,35,0,32,32.0
30,30,1,56,28.0
41,41,1,56,28.0


In [17]:
def deltas(l):
    return list(round(pd.Series(l[1:len(l) + 1]) - pd.Series(l[0:len(l)]), 1))

In [17]:
deltas(list(df_avgunbrk.avg_unbroken))

[-13.0, -8.1, -1.2, -2.0, nan]